In [14]:
import numpy as np
from tqdm.auto import tqdm
import scipy.sparse.linalg
from scipy.sparse.linalg import LinearOperator,aslinearoperator
from functorch import jvp,vjp
from math import prod
import torch
from opt_einsum import contract
from scipy.sparse.linalg import eigsh,svds

In [15]:
def get_linearized_cylinder(T0):
    dimT=prod(T0.shape)
    pbar=tqdm()
    def matvec(v):
        shape0=v.shape
        v=v.reshape(T0.shape)
        v=contract('iIab,jJbc,kKcd,lLda,IJKL->ijkl',T0,T0,T0,T0,v).reshape(-1)
        pbar.update(1)
        return v.reshape(shape0)
    def rmatvec(v):
        shape0=v.shape
        v=v.reshape(T0.shape)
        v=contract('iIab,jJbc,kKcd,lLda,ijkl->IJKL',T0,T0,T0,T0,v.conj()).conj().reshape(-1)
        pbar.update(1)
        return v.reshape(shape0)
    return LinearOperator(shape=(dimT,dimT),matvec=matvec,rmatvec=rmatvec)

In [16]:
T=np.random.randn(6,6,6,6)

M=get_linearized_cylinder(T)

0it [00:00, ?it/s]

In [ ]:
svds(M,k=16,ncv=32,return_singular_vectors=False)